# MaaS_General_Challenge

## Problem analysis

### Problem statement

Eliminate overlapped convex hulls

* Load convex_hulls.json.
* Some of the convex hulls stored in convex_hulls.json are overlapped.
* Eliminate the overlapped convex hulls based on the following rule.
    * Eliminate the convex hull if it has more than 50% of its own area overlapped with another convex hull (pairwise area overlap).
* Generate result_convex_hulls.json which stores only the remaining (non-eliminated) convex hulls. The format should be the same as convex_hulls.json.
* Authorized program language is only C++.
    * Follow a coding style guide which you are using.
        * E.g. Google C++ Style Guide
    * Mention the style guide as a comment on the top of your program code.
        * // This code follows Google C++ Style Guide.
* You are allowed to use JSON library to read/write a json file.
    * E.g. https://nlohmann.github.io/json/.
* You are allowed to use the std library in C++.
* Do not use any other libraries, especially to calculate area or intersection.
* Put appropriate comments for the reviewer to understand your code easily.
* Submit the following files:
    * your code and the code for the json library you used
    * your result_convex_hulls.json
    * Instruction to build and run your code so that we can reproduce your result using your submission

### Reformulation of the problem statement for clarity

#### Task

Eliminate overlapped convex hulls

* Load convex_hulls.json.
* Some of the convex hulls stored in convex_hulls.json are overlapped.
* Eliminate the overlapped convex hulls based on the following rule.
    * Eliminate the convex hull if it has more than 50% of its own area overlapped with another convex hull (pairwise area overlap).
* Generate result_convex_hulls.json which stores only the remaining (non-eliminated) convex hulls.

#### Constraints

* The format should be the same as convex_hulls.json.
* Authorized program language is only C++.
    * Follow a coding style guide which you are using.
        * E.g. Google C++ Style Guide
    * Mention the style guide as a comment on the top of your program code.
        * // This code follows Google C++ Style Guide.
* You are allowed to use JSON library to read/write a json file.
    * E.g. https://nlohmann.github.io/json/.
* You are allowed to use the std library in C++.
* Do not use any other libraries, especially to calculate area or intersection.
* Put appropriate comments for the reviewer to understand your code easily.

#### Submission

* Submit the following files:
    * your code and the code for the json library you used
    * your result_convex_hulls.json
    * Instruction to build and run your code so that we can reproduce your result using your submission

### Initial analysis (Visualization of the problem)

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import math
from collections import namedtuple
from inspect import signature

# Increase the size of Jupyter plots
plt.rcParams['figure.figsize'] = [20, 10]

json_filepath = "./convex_hulls.json"

def setAxis():
    plt.xlim([-50, 20])
    plt.ylim([-30, 40])

def get_color(idx):
    color_palette = [color for color in mcolors.TABLEAU_COLORS]
    return color_palette[idx % len(color_palette)]

def plot_convex_hull(convex_hull, *args, **kwargs):
    plt.plot(np.append(convex_hull[:,0], [[convex_hull[0, 0]]], axis=0), np.append(convex_hull[:,1], [[convex_hull[0,1]]], axis=0), *args, **kwargs)

with open(json_filepath) as f:
    data = json.load(f)

convex_hulls = []
for convex_hull in data["convex hulls"]:
    points = []
    for apex in convex_hull["apexes"]:
        points.append([apex['x'], apex['y']])
    points = np.matrix(points)
    convex_hulls.append(points)
    plot_convex_hull(points, color=get_color(convex_hull["ID"]))
    plt.text(points[0, 0], points[0, 1], convex_hull["ID"])

plt.axis("square")
setAxis()
plt.grid()

## Part 1 : Intersection of convex hulls

### Research of algorithms

After a quick research on intersection of convex polygon, I fould the following:  
https://tildesites.bowdoin.edu/~ltoma/teaching/cs3250-CompGeom/spring17/Lectures/cg-convexintersection.pdf

This document seems to refer to the algorithm described in the following paper:  
https://www.cs.jhu.edu/~misha/Spring16/ORourke82.pdf

The algorithm is described as simple to implement and run in linear time.  
So I decided to proceed with this algorithm.

### Algorithm (extract from ORourke82)

The algorithm is describe in pseudo-code as follows:

input: $P$ and $Q$  
output: $P \cap Q$  

**begin** {main}  
Choose $\mathbf{p}$ and $\mathbf{q}$ arbitrarily.  
**repeat**  
> {Check to see if $\mathbf{\dot{p}}$ and $\mathbf{\dot{q}}$ intersect}  
> **if** $\mathbf{\dot{p}}$ and $\mathbf{\dot{q}}$ intersect **then**  

>> **if** this intersection is the same as the first intersection **then** halt  
>> **else** output the point of intersection and  

>>> {set inside} **if** $\mathbf{p} \in hp(\mathbf{\dot{q}})$ **then** $inside \leftarrow “P”$ else $inside \leftarrow “Q“$;  

>>{Advance either $\mathbf{p}$ or $\mathbf{q}$.}  
>> **if** $\mathbf{\dot{q}} \times \mathbf{\dot{p}} \geq O$ **then**  

>>> **if** $\mathbf{p} \in hp(\mathbf{\dot{q}})$ **then** (advance $\mathbf{q}$) **else** (advance $\mathbf{p}$)  

>> **else** {$\mathbf{\dot{q}} \times \mathbf{\dot{p}} \lt 0$}  

>>> **if** $\mathbf{q} \in hp(\mathbf{\dot{p}})$ **then** (advance $\mathbf{p}$) **else** (advance $\mathbf{q}$);  

**until** repeat has executed more than $2(\lvert P \rvert + \lvert Q \rvert)$ times;  
{either $P \cap Q = \emptyset$ or $P \supseteq Q$ or $P \subseteq Q$}  
Choose $\mathbf{p}$ and $\mathbf{q}$ arbitrarily.  
**if** $\mathbf{p} \in Q$ **then** output $P$ **else** **if** $\mathbf{q} \in P$ **then** output $Q$ **else** output $\emptyset$ ;  
**end** {main}  


advance $\mathbf{p}$:  
> output $\mathbf{p}$ **if** $inside = “P“$;  
> $p \leftarrow P_+$;  

advance $\mathbf{q}$:  
> output $\mathbf{q}$ **if** $inside =“Q”$;  
> $q \leftarrow q_+$;  

$hp(\mathbf{\dot{p}}) = \{\mathbf{x} : \mathbf{\dot{p}} \times (\mathbf{x} - \mathbf{p_-}) \geq 0\}$

After implementing the algorithm in C++, I wrote python bindings to the C++ code.  
Python bindings doesn't affect the original code.  
They are just wrapper code that call the actual C++ code.  
The reason I wrote python bindings is so that I could visualize my results using this jupyter notebook.

### Visualization of results (Convex Hull intersection and area calculation)

In [ ]:
from convex_hull_filtering import *
def plot_pair(convex_hulls, idx1, idx2):
    [inter, interConvexHull, area] = intersection(convex_hulls[idx1], convex_hulls[idx2])
    plot_convex_hull(convex_hulls[idx1], color=get_color(idx1))
    plot_convex_hull(convex_hulls[idx2], color=get_color(idx2))
    if inter:
        interConvexHull = np.matrix(interConvexHull)
        plot_convex_hull(interConvexHull, linewidth=5, color="red")
    area1 = getArea(convex_hulls[idx1])
    area2 = getArea(convex_hulls[idx2])
    plt.title(f"{idx1} ({area1:.2f}) & {idx2} ({area2:.2f}) / Inter area = {area:.2f}")
    plt.axis("square")
    plt.grid()

plt.subplot(3,2,1)
plot_pair(convex_hulls, 1, 2)

plt.subplot(3,2,2)
plot_pair(convex_hulls, 3, 4)

plt.subplot(3,2,3)
plot_pair(convex_hulls, 5, 6)

plt.subplot(3,2,4)
plot_pair(convex_hulls, 7, 8)

plt.subplot(3,3,7)
plot_convex_hull(convex_hulls[11], color=get_color(11))
plot_pair(convex_hulls, 9, 10)

plt.subplot(3,3,8)
plot_convex_hull(convex_hulls[10], color=get_color(10))
plot_pair(convex_hulls, 9, 11)

plt.subplot(3,3,9)
plot_convex_hull(convex_hulls[9], color=get_color(9))
plot_pair(convex_hulls, 10, 11)

### Checking degenerate cases

Checking the following cases:
* the two convex hull share one edge
* the two convex hull intersect on a single point
* the two convex hull partially overalap on one edge
* the two convex hull share one edge (different starting point)

In [ ]:
pointsA = np.matrix([[0.0, 0.0], [1.0, 0.0], [1.0, 1.0]])
pointsB = np.matrix([[1.0, 0.0], [1.0, 1.0], [0.0, 1.0]])
pointsC = np.matrix([[1.0, 0.5], [2.0, 0.5], [2.0, 1.5]])
pointsD = np.matrix([[0.5, 0.0], [1.5, 0.0], [1.5, 1.0]])
pointsE = np.matrix([[0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
plt.subplot(2, 2, 1)
plot_pair([pointsA, pointsB], 0, 1)
print(intersection(pointsA, pointsB))
plt.subplot(2, 2, 2)
plot_pair([pointsA, pointsC], 0, 1)
print(intersection(pointsA, pointsC))
plt.subplot(2, 2, 3)
plot_pair([pointsA, pointsD], 0, 1)
print(intersection(pointsA, pointsD))
plt.subplot(2, 2, 4)
plot_pair([pointsA, pointsE], 0, 1)
print(intersection(pointsA, pointsE))

Checking with different direction to travel ie clockwise or counter-clockwise

In [ ]:
pointsA = []
for theta in np.linspace(1, 1 + 2 * math.pi, 10):
    l = 5
    pointsA.append([l * math.cos(theta), l * math.sin(theta)])
pointsA = np.matrix(pointsA)

pointsB = []
for theta in np.linspace(0, -2 * math.pi, 10):
    l = 5
    pointsB.append([l * math.cos(theta), l * math.sin(theta)])
pointsB = np.matrix(pointsB)

plot_pair([pointsA, pointsB], 0, 1)

[inter, interConvexHull, area] = intersection(pointsA, pointsB)

## Part 2 : Collision detection

### Research of algorithms

After some research, I found that there is a multitude of algorithms available.  
For the task requested, I decided to try to use R-tree algorithms  
Again, there is many variants of R-tree but after considering the short time (1 week) for this assigment,  
I decided to implement the original R-tree algorithm following the original paper:  
http://www-db.deis.unibo.it/courses/SI-LS/papers/Gut84.pdf

Also I decided to limit myself to only the search and insertion cases (ie not deletion) due to the short time limitation

### Algorithm (extract from Gut84)

Algorithm **Search**  
Given an R-tree whose root node is $T$, find all index records whose rectangles overlap a search rectangle $S$  

Sl [Search subtrees]  
If $T$ is not a leaf, check each entry $E$ to deterrmine whether $E I$ overlaps $S$  
For all overlapping entries, invoke **Search** on the tree whose root node is pointed to by $Ep$  

S2 [Search leaf node]  
If $T$ is a leaf, check all entries $E$ to determine whether $EI$ overlaps $S$ 
If so, $E$ is a qualifying record



Algorithm **Insert**  
Insert a new index entry $E$ into an R-tree  

I1 [Find position for new record]  
Invoke **ChooseLeaf** to select a leaf node $L$ in which to place $E$

I2 [Add record to leaf node]  
If $L$ has room for another entry, install $E$  
Otherwise invoke **SplitNode** to obtain $L$ and $LL$ containing $E$ and all the old entries of $L$  

I3 [Propagate changes upward]  
Inovke **AdjustTree** on $L$, also passing $LL$ if a split was performed  

I4 [Grow tree taller]  
If node split propagation caused the root to split, create a new root whose children are the two resulting nodes

### Visualization of results

In [ ]:
entries = []

def plot_bb(bb, *args, **kwargs):
    plt.plot([bb[0], bb[2], bb[2], bb[0], bb[0]], [bb[1], bb[1], bb[3], bb[3], bb[1]], *args, **kwargs)

def traverseTree(node, level):
    bb = node["bb"]
    linewidth = 25 - (10 * level)
    color = get_color(level)
    if len(node["children"]) == 0:
        linewidth = 1
        color = get_color(node["value"])
    plot_bb(bb, linewidth=linewidth, color=color)
    for child in node["children"]:
        traverseTree(child, level + 1)

def plotEachLevel(node, level, plot_level):
    bb = node["bb"]
    linewidth = 1
    color = get_color(level)
    if len(node["children"]) == 0:
        color = get_color(node["value"])
    if level == plot_level:
        plot_bb(bb, linewidth=linewidth, color=color)
    for child in node["children"]:
        plotEachLevel(child, level + 1, plot_level)

plt.figure()
for i, convex_hull in enumerate(convex_hulls):
    plot_convex_hull(convex_hull, color=get_color(i))
    bb = boundingBox(convex_hull)
    entries.append([i] + bb)
entries = np.matrix(entries)
tree = insertEntry(1, 3, entries)
traverseTree(tree, 0)
plt.axis("square")
plt.grid(True)
setAxis()

plt.figure()
for i in range(4):
    plt.subplot(1,4, i + 1)
    plotEachLevel(tree, 0, i)
    plt.axis("square")
    plt.grid(True)
    setAxis()

## Part 3 : Putting everything together

In [ ]:
res = searchOverlaps(1, 3, entries, entries[:,1:5])
res